Para esse caso de estudo, discutirei uma possível solução para o problema apresentado.

Alguns pontos observados inicialmente, ao olhar para o formato de dado usado como exemplo:

- O relatório/fonte de dados está armazenado no que assumo ser uma planilha de Excel/google sheets. Isso é um ponto positivo, pois a maior parte das ferramentas de importação de dados tem uma integração razoável com esse tipo de arquivo.
- Considero a fonte de dados como tendo uma certa estrutura, mesmo que o formato não seja o mais ideal para análises de dados.
- Temos alguns dados "soltos", que são relevantes àquela coleta de dados, mas não se encaixam em nenhuma tabela. Isso indica que precisaremos de uma classe de dados mais complexa, algo que armazene todos os detalhes presentes na metade superior do relatório, além das tabelas presentes na metade inferior.


Dadas essas observações, já é possível tracejar e implementar uma possível solução para o manuseio dos dados.

Antes disso, porém, gostaria de destacar alguns pontos que irei "assumir" para esse estudo de caso, em nome da objetividade e clareza da solução.

- Suposição 1: Todos as células presentes no relatório são fixas, isso é, entre um relatório e outro, não ocorrerá de uma descrição de dado e seu valor atrelado mudarem de lugar numa frequência que necessite de retrabalho constante.
- Suposição 2: Todos os relatórios são iguais. Além das células não mudarem de lugar, todos os relatórios possuirão as mesmas células e os mesmos tipos de dados contidos nelas. Dessa forma, é possível implementar uma solução genérica que funcione em qualquer relatório enviado.
- Suposição 3: Não é necessário separar as tabelas de movimentos/contagem de veículos. Pude notar por meio da análise prévia do relatório que existe uma separação entre os períodos de coleta (manhã, tarde e noite) e até entre os intervalos do mesmo período. O dado do intervalo em si é, claro, muito importante, mas é possível armazenar todos os períodos e intervalos em apenas uma estrutura de dados, sem impactar a capacidade de fazermos consultas a períodos e intervalos específicos. Para ser franco, é possível que tenha um impacto negativo na performance, pois teríamos tabelas maiores, mas é um custo ínfimo se comparado a praticidade e facilidade de implementação ao trabalharmos com apenas uma tabela.
- Suposição 4: Cada conjunto de colunas [Horário] + [Auto, Bus, Cam., Moto, Bici. UVP] corresponde a apenas 1 movimento. Para estruturação no formato de tabela, faremos com que o movimento seja mais uma coluna.

Após todas as observações e suposições detalhadas, podemos seguir adiante com os detalhes da solução:

1. O coração da solução serão os objetos do tipo dataframe, muito utilizados para representar dados colunares.
2. Comumente, a biblioteca de python "Pandas" é utilizada para manuseio de dataframes, mas para esse exercício, utilizarei a biblioteca "Polars" que é mais nova (e por isso menos "estável" e atestada, porém tem performance muito superior se comparada à "Pandas". Fonte: https://h2oai.github.io/db-benchmark/). Além disso, para importação direta de datas, será necessário plugar a lib "pyarrow" ao "Polars".
3. Como citado na etapa de observação, temos alguns dados que não cabem na visão de dataframe construída até aqui, como "Ponto de Pesquisa" ou "Endereço". É possível incorporar "data da pesquisa" na tabela, se for necessário, mas nesse ponto, não está clara a necessidade disso. Sendo assim, faz-se necessário uma classe que armazene os dados gerais e dataframe. Essa abordagem talvez não faça sentido num Python Notebook como esse, mas tem suas vantagens quando aplicada à produção de um projeto.
4. Será necessário um script que extraia todos os dados relevantes da planilha e os use para compor os objetos estruturados criados em Python. Para esse fim, a biblioteca "openpyxl" será usada. Ela permite que cada célula individual de um arquivo .xlsx seja acessada.

Para começar, instalamos e importamos as dependências necessárias.

In [1]:
# These extra bits are fail-safes to ensure the packages are being installed in
# the jupyter kernel that is actually, currently, running

import sys
!{sys.executable} -m pip install polars
!{sys.executable} -m pip install openpyxl
!{sys.executable} -m pip install pyarrow
!{sys.executable} -m pip install plotly

import polars as pl
import openpyxl
import plotly.express as px
from datetime import datetime, time

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


Implementação do scrip extrator de dados.

De forma geral, o extrator funciona acessando células pré-determinadas onde se encontram as informações gerais,
mas extrai informações de tabela para criação do DF (DataFrame) de forma mais dinâmica. Vale destacar a separação de intervalos de tempo em 2 colunas, interval_start e interval_end, ambas já processadas com o tipo datetime.Time (ou pl.Time, caso no DF) para maior facilidade de consultas, e também o fato de que o dado para "Movimento" foi transferido para uma coluna.

Na planilha passada como exemplo, notei que a cada bloco de intervalos era possível encontrar um valor entre um segmento de tabela e outro, na coluna UVP. Imediatamente imaginei que seria a somo dos UVP's acima, mas percebi que a conta nem sempre batia, mas sempre por pouco (1 pra mais ou pra menos). Sendo assim, assumi que era uma flutuação decorrente de algum arredondamento com operações em números decimais que não estava visível no exemplo, e decidi não registrar esse dado, sendo que sempre é possível somar os UVP's de um bloco de coletas para termos o dado novamente.

In [2]:
def xlsx_extractor(file_path: str):
    """
    Receives a .xlsx file path.
    return the data extracted from the sheet.    
    """
    def extract_hour_from_interval(date_interval: str):
        """
        Receives a string in the format:
        "hh:mm às hh:mm"
        Returns the 2 hours in datetime.time format.
        """
        start_time = date_interval[:5]
        end_time = date_interval[9:]

        start_time = datetime.strptime(start_time, '%H:%M').time()
        end_time = datetime.strptime(end_time, '%H:%M').time()

        return start_time, end_time

    workbook = openpyxl.load_workbook(file_path)
    sheet = workbook.active

    # Getting general data

    research_point = sheet['B2'].value
    address = sheet['B3'].value
    research_date = sheet['B6'].value
    researh_date_weekday = sheet['C6'].value
    period_rows = [5, 8, 13]

    intervals_fhp = {}
    for period_row in period_rows:
        period = sheet.cell(row=period_row, column=1).value
        period = period[9:]

        row_range = 1
        if period == "Tarde":
            row_range = 3

        for i in range(row_range):
            index = i + 1
            interval = sheet.cell(row=period_row+index, column=5)
            interval_start, interval_end = extract_hour_from_interval(interval.value)
            fhp = sheet.cell(row=period_row+index, column=7).value
            key = (period, interval_start, interval_end)
            intervals_fhp[key] = fhp

    # Getting table data

    table_data = {
        "interval_start": [],
        "interval_end": [],
        "movimento": [],
        "Auto": [],
        "Bus": [],
        "Cam.": [],
        "Moto": [],
        "Bici.": [],
        "UVP": [],
    }

    # Iterate through rows and movimentos to get count data.

    movimentos_columns = [2, 8, 14]

    for movimento_column in movimentos_columns:
        movimento = sheet.cell(row=16, column=movimento_column).value

        # getting interval separately
        for row in sheet.iter_rows(min_row=19, min_col=1, max_row=32, max_col=1, values_only=True):
            interval = row[0]
            if interval is not None:
                interval_start, interval_end = extract_hour_from_interval(interval)
                table_data["interval_start"].append(interval_start)
                table_data["interval_end"].append(interval_end)

        min_col = movimento_column
        max_col = min_col+5

        for row in sheet.iter_rows(min_row=19, min_col=min_col, max_row=32, max_col=max_col, values_only=True):
            if row[0] is not None:
                table_data["movimento"].append(int(movimento))
                table_data["Auto"].append(int(row[0]))
                table_data["Bus"].append(int(row[1]))
                table_data["Cam."].append(int(row[2]))
                table_data["Moto"].append(int(row[3]))
                table_data["Bici."].append(int(row[4]))
                table_data["UVP"].append(int(row[5]))

    columns = [
        ("interval_start", pl.Time),
        ("interval_end", pl.Time),
        ("movimento", pl.Int64),
        ("Auto", pl.Int64),
        ("Bus", pl.Int64),
        ("Cam.", pl.Int64),
        ("Moto", pl.Int64),
        ("Bici.", pl.Int64),
        ("UVP", pl.Float64),
    ]
    dataframe = pl.DataFrame(table_data, columns=columns)

    return {
        "research_point": research_point,
        "address": address,
        "research_date": research_date,
        "researh_date_weekday": researh_date_weekday,
        "intervals_fhp": intervals_fhp,
        "dataframe": dataframe
    }

testando extrator e visualizando o objeto dataframe criado.

In [3]:
file_path = "./case_study_assets/TPF-test.xlsx"
extracted_data = xlsx_extractor(file_path)
for key in extracted_data:
    if key != "dataframe":
        print(key, ":", extracted_data[key])

df = extracted_data['dataframe']
print(df.head())

research_point : 4.0
address : Rua tal
research_date : 2022-08-01 00:00:00
researh_date_weekday : Segunda
intervals_fhp : {('Manhã', datetime.time(7, 30), datetime.time(8, 30)): 0.87, ('Tarde', datetime.time(12, 0), datetime.time(13, 0)): 0.6, ('Tarde', datetime.time(14, 30), datetime.time(15, 30)): 0.67, ('Tarde', datetime.time(17, 0), datetime.time(18, 0)): 0.62, ('Noite', datetime.time(20, 0), datetime.time(21, 0)): 0.71}
shape: (5, 9)
┌────────────────┬──────────────┬───────────┬──────┬─────┬──────┬──────┬───────┬───────┐
│ interval_start ┆ interval_end ┆ movimento ┆ Auto ┆ ... ┆ Cam. ┆ Moto ┆ Bici. ┆ UVP   │
│ ---            ┆ ---          ┆ ---       ┆ ---  ┆     ┆ ---  ┆ ---  ┆ ---   ┆ ---   │
│ time           ┆ time         ┆ i64       ┆ i64  ┆     ┆ i64  ┆ i64  ┆ i64   ┆ f64   │
╞════════════════╪══════════════╪═══════════╪══════╪═════╪══════╪══════╪═══════╪═══════╡
│ 07:30:00       ┆ 07:45:00     ┆ 1         ┆ 131  ┆ ... ┆ 1    ┆ 16   ┆ 1     ┆ 154.0 │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌

Criação de uma classe que abstrai a extração e manuseio dos dados.

Em produção, incluímos o extrator na classe.

E também criamos algumas funções que abstraem consultas comuns que podem ser interessantes, como "Busque todas as contagens para o movimento 1", ou "Busque todas as contagens que aconteceram durante esse intervalo de tempo".


In [4]:
class VehicleCount:
    def __init__(self, sheet_file_path: str):
        """ Constructor """
        extracted_data = xlsx_extractor(sheet_file_path)
        self.research_point = extracted_data['research_point']
        self.address = extracted_data['address']
        self.research_date = extracted_data['research_date']
        self.researh_date_weekday = extracted_data['researh_date_weekday']
        self.intervals_fhp = extracted_data['intervals_fhp']
        self.dataframe = extracted_data['dataframe']

    def xlsx_extractor(file_path: str):
        """
        Receives a .xlsx file path.
        return the data extracted from the sheet.    
        """
        def extract_hour_from_interval(date_interval: str):
            """
            Receives a string in the format:
            "hh:mm às hh:mm"
            Returns the 2 hours in datetime.time format.
            """
            start_time = date_interval[:5]
            end_time = date_interval[9:]

            start_time = datetime.strptime(start_time, '%H:%M').time()
            end_time = datetime.strptime(end_time, '%H:%M').time()

            return start_time, end_time

        workbook = openpyxl.load_workbook(file_path)
        sheet = workbook.active

        # Getting general data

        research_point = sheet['B2'].value
        address = sheet['B3'].value
        research_date = sheet['B6'].value
        researh_date_weekday = sheet['C6'].value
        period_rows = [5, 8, 13]

        intervals_fhp = {}
        for period_row in period_rows:
            period = sheet.cell(row=period_row, column=1).value
            period = period[9:]

            row_range = 1
            if period == "Tarde":
                row_range = 3

            for i in range(row_range):
                index = i + 1
                interval = sheet.cell(row=period_row+index, column=5)
                interval_start, interval_end = extract_hour_from_interval(interval.value)
                fhp = sheet.cell(row=period_row+index, column=7).value
                key = (period, interval_start, interval_end)
                intervals_fhp[key] = fhp

        # Getting table data

        table_data = {
            "interval_start": [],
            "interval_end": [],
            "movimento": [],
            "Auto": [],
            "Bus": [],
            "Cam.": [],
            "Moto": [],
            "Bici.": [],
            "UVP": [],
        }

        # Iterate through rows and movimentos to get count data.

        movimentos_columns = [2, 8, 14]

        for movimento_column in movimentos_columns:
            movimento = sheet.cell(row=16, column=movimento_column).value

            # getting interval separately
            for row in sheet.iter_rows(min_row=19, min_col=1, max_row=32, max_col=1, values_only=True):
                interval = row[0]
                if interval is not None:
                    interval_start, interval_end = extract_hour_from_interval(interval)
                    table_data["interval_start"].append(interval_start)
                    table_data["interval_end"].append(interval_end)

            min_col = movimento_column
            max_col = min_col+5

            for row in sheet.iter_rows(min_row=19, min_col=min_col, max_row=32, max_col=max_col, values_only=True):
                if row[0] is not None:
                    table_data["movimento"].append(int(movimento))
                    table_data["Auto"].append(int(row[0]))
                    table_data["Bus"].append(int(row[1]))
                    table_data["Cam."].append(int(row[2]))
                    table_data["Moto"].append(int(row[3]))
                    table_data["Bici."].append(int(row[4]))
                    table_data["UVP"].append(int(row[5]))

        columns = [
            ("interval_start", pl.Time),
            ("interval_end", pl.Time),
            ("movimento", pl.Int64),
            ("Auto", pl.Int64),
            ("Bus", pl.Int64),
            ("Cam.", pl.Int64),
            ("Moto", pl.Int64),
            ("Bici.", pl.Int64),
            ("UVP", pl.Float64),
        ]
        dataframe = pl.DataFrame(table_data, columns=columns)

        return {
            "research_point": research_point,
            "address": address,
            "research_date": research_date,
            "researh_date_weekday": researh_date_weekday,
            "intervals_fhp": intervals_fhp,
            "dataframe": dataframe
        }    

    def get_counts_in_movimento(self, movimento: int):
        """
        Receives an int that represents the movimento number.
        returns a sliced dataframe with only records that belong to the movimento.
        """
        filtered_df = self.dataframe.filter(pl.col("movimento") == movimento)
        return filtered_df

    def get_counts_in_interval(self, start_time: str, end_time: str):
        """
        Receives start and end times that represent the interval.
        returns a sliced dataframe with only records that belong to the interval.
        """
        date_format = '%H:%M'
        start_time = pl.lit(start_time).str.strptime(pl.Time, fmt=date_format)
        end_time = pl.lit(end_time).str.strptime(pl.Time, fmt=date_format)
        filtered_df = self.dataframe.filter(
            (pl.col("interval_start") >= start_time) & (pl.col("interval_end") <= end_time)
        )
        return filtered_df

    def dump_dataframe_to_csv(self, output_csv_file_path: str):
        """
        Receives an output file path.
        Dumps the contents of the dataframe to a .csv file and returns the file path.
        """
        self.dataframe.write_csv(output_csv_file_path, sep=";")
        return output_csv_file_path

    def get_total_uvp_per_movimentos(self):
        """
        Returns a df containing the movimentos and the total UVP's for that movimento.
        """
        q = (
            self.dataframe.lazy()
            .groupby(['movimento'])
            .agg([pl.sum('UVP')])
        )
        return q.collect()

In [5]:
vc = VehicleCount(file_path)

In [6]:
print("Ponto de Pesquisa:", vc.research_point)
print("Endereço:", vc.address)

Ponto de Pesquisa: 4.0
Endereço: Rua tal


In [7]:
print("Contagens do movimento 1:")
filtered_df = vc.get_counts_in_movimento(1)
print(filtered_df)

uvp_sum = pl.sum(filtered_df.get_column("UVP"))
print("Soma dos UVP's do movimento 1:", uvp_sum)


Contagens do movimento 1:
shape: (12, 9)
┌────────────────┬──────────────┬───────────┬──────┬─────┬──────┬──────┬───────┬───────┐
│ interval_start ┆ interval_end ┆ movimento ┆ Auto ┆ ... ┆ Cam. ┆ Moto ┆ Bici. ┆ UVP   │
│ ---            ┆ ---          ┆ ---       ┆ ---  ┆     ┆ ---  ┆ ---  ┆ ---   ┆ ---   │
│ time           ┆ time         ┆ i64       ┆ i64  ┆     ┆ i64  ┆ i64  ┆ i64   ┆ f64   │
╞════════════════╪══════════════╪═══════════╪══════╪═════╪══════╪══════╪═══════╪═══════╡
│ 07:30:00       ┆ 07:45:00     ┆ 1         ┆ 131  ┆ ... ┆ 1    ┆ 16   ┆ 1     ┆ 154.0 │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌┼╌╌╌╌╌┼╌╌╌╌╌╌┼╌╌╌╌╌╌┼╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌┤
│ 07:45:00       ┆ 08:00:00     ┆ 1         ┆ 142  ┆ ... ┆ 0    ┆ 33   ┆ 0     ┆ 174.0 │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌┼╌╌╌╌╌┼╌╌╌╌╌╌┼╌╌╌╌╌╌┼╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌┤
│ 08:00:00       ┆ 08:15:00     ┆ 1         ┆ 118  ┆ ... ┆ 0    ┆ 17   ┆ 2     ┆ 155.0 │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌┼╌╌╌╌╌┼╌╌╌╌╌╌┼╌╌╌╌

In [8]:
start_time = "07:45"
end_time = "08:15"
print(f"Contagens entre o intervalo {start_time} às {end_time}")
filtered_df = vc.get_counts_in_interval(start_time, end_time)
print(filtered_df)

Contagens entre o intervalo 07:45 às 08:15
shape: (6, 9)
┌────────────────┬──────────────┬───────────┬──────┬─────┬──────┬──────┬───────┬───────┐
│ interval_start ┆ interval_end ┆ movimento ┆ Auto ┆ ... ┆ Cam. ┆ Moto ┆ Bici. ┆ UVP   │
│ ---            ┆ ---          ┆ ---       ┆ ---  ┆     ┆ ---  ┆ ---  ┆ ---   ┆ ---   │
│ time           ┆ time         ┆ i64       ┆ i64  ┆     ┆ i64  ┆ i64  ┆ i64   ┆ f64   │
╞════════════════╪══════════════╪═══════════╪══════╪═════╪══════╪══════╪═══════╪═══════╡
│ 07:45:00       ┆ 08:00:00     ┆ 1         ┆ 142  ┆ ... ┆ 0    ┆ 33   ┆ 0     ┆ 174.0 │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌┼╌╌╌╌╌┼╌╌╌╌╌╌┼╌╌╌╌╌╌┼╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌┤
│ 08:00:00       ┆ 08:15:00     ┆ 1         ┆ 118  ┆ ... ┆ 0    ┆ 17   ┆ 2     ┆ 155.0 │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌┼╌╌╌╌╌┼╌╌╌╌╌╌┼╌╌╌╌╌╌┼╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌┤
│ 07:45:00       ┆ 08:00:00     ┆ 2         ┆ 47   ┆ ... ┆ 0    ┆ 8    ┆ 0     ┆ 49.0  │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌┼╌

In [9]:
output_file_path = "./case_study_assets/output_csv.csv"
vc.dump_dataframe_to_csv(output_file_path)

'./case_study_assets/output_csv.csv'

Visualização

In [11]:
r = vc.get_total_uvp_per_movimentos()

barchart = px.bar(
    r.to_pandas(),
    x = "movimento",
    y = "UVP"
)

barchart.show()

# Considerações finais
- Nesse ponto da visualização, concluí que foi um erro ter deixado o tipo de movimento como inteiro. De fato, o movimento provavelmente é uma "label", e não um valor. Creio, agora, que não faria sentido fazermos operações aritméticas com os números dos movimentos, sendo assim, o ideal é que deixássemos ele como uma string mesmo, e não como um inteiro. Podemos ver uma característica dos inteiros no eixo x do gráfico, que é a criação de uma range de movimentos [0, 0.5, 1, 1.5, 2, 2.5]. Mas um movimento 0.5 ou 1.5 provavelmente não faz o menor sentido.

- Além disso, senti falta de uma coluna "intervalo", assimo como existia antes de separarmos. Talvez seja interessante manter essa coluna como string mesmo, além das colunas "interval_start" e "interval_end", para fins de agrupamento.

- Talvez tivesse sido uma boa ideia, além de guardar apenas as horas dos intervalos, armazenar o dia da coleta. Poderia até ser uma coluna só, fazendo uso do tipo datetime. Aumentaria as possibilidades de queries possíveis.

- Existe um grande número de queries possíveis de serem feitas com o dataframe. Fiz algumas que são comuns a qualquer análise estatística, mas não explorei muito esse aspecto em nome da objetividade.

- Não consegui replicar as imagens presentes no exemplo de relatório. Assumindo que elas representem gráficos que não possamos refazer, isso poderia ser um problema. Acredito que exista, sim, uma maneira de acessar essas imagens, mas como nem fui capaz de replicar no meu .xlsx de teste, não explorei muito nesse sentido.

- Não explorei uma maneira de acessar dados contidos em "abas" diferentes da planilha, mas a lib "openpyxl" fornece uma interface para mudar qual aba da planilha está sendo acessada, tornando trivial a adaptação da solução para incluir essa feature. 

- Por fim, o plotly não se comportou bem com o Polars, o que é de esperar, já que é uma tecnologia mais recente e ainda está sendo adotada mundo afora. Para plotar o gráfico de barras acima, foi necessário converter de Polars para Pandas, possivelmente derrotando o esforço de ter adotado o Polars de início (Dito isso, ainda assim creio valer a pena manter o Polars e simplesmente converter para Pandas quando necessário, pois o ganho de performance no manuseio dos dados é possivelmente maior do que a perda das conversões).